***GENERATED CODE FOR gettingdata PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Category', 'transformation_label': 'String Indexer'}], 'feature': 'Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '7995', 'mean': '', 'stddev': '', 'min': 'Furniture', 'max': 'Technology', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Category'}, {'feature_label': 'Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '', 'stddev': '', 'min': 'Aberdeen', 'max': 'Yuma', 'missing': '0', 'distinct': '503'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Country', 'transformation_label': 'String Indexer'}], 'feature': 'Country', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '', 'stddev': '', 'min': 'United States', 'max': 'United States', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Country'}, {'feature_label': 'Country', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Country')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer ID', 'transformation_label': 'String Indexer'}], 'feature': 'Customer ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '', 'stddev': '', 'min': 'AA-10315', 'max': 'ZD-21925', 'missing': '0', 'distinct': '715'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer ID'}, {'feature_label': 'Customer ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer Name', 'transformation_label': 'String Indexer'}], 'feature': 'Customer Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '', 'stddev': '', 'min': 'Aaron Bergman', 'max': 'Zuschuss Donatelli', 'missing': '0', 'distinct': '715'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer Name'}, {'feature_label': 'Customer Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Order Date', 'transformation_label': 'String Indexer'}], 'feature': 'Order Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '', 'stddev': '', 'min': '1/1/2016', 'max': '9/9/2016', 'missing': '0', 'distinct': '1196'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Order Date'}, {'feature_label': 'Order Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Order Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Order ID', 'transformation_label': 'String Indexer'}], 'feature': 'Order ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '', 'stddev': '', 'min': 'CA-2011-100006', 'max': 'US-2014-169551', 'missing': '0', 'distinct': '4092'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Order ID'}, {'feature_label': 'Order ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Order ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product ID', 'transformation_label': 'String Indexer'}], 'feature': 'Product ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '', 'stddev': '', 'min': 'FUR-BO-10000112', 'max': 'TEC-PH-10004977', 'missing': '0', 'distinct': '1821'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product ID'}, {'feature_label': 'Product ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product Name', 'transformation_label': 'String Indexer'}], 'feature': 'Product Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '', 'stddev': '', 'min': '"""While you Were Out"" Message Book', 'max': 'netTALK DUO VoIP Telephone Service', 'missing': '0', 'distinct': '1803'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product Name'}, {'feature_label': 'Product Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Region', 'transformation_label': 'String Indexer'}], 'feature': 'Region', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '', 'stddev': '', 'min': ' 10/Pack"', 'max': 'West', 'missing': '0', 'distinct': '43'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Region'}, {'feature_label': 'Region', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Region')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Row ID', 'transformation_label': 'String Indexer'}], 'feature': 'Row ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '4983.42', 'stddev': '2891.66', 'min': ' 1040 sheets"', 'max': 'West', 'missing': '0', 'distinct': '7754'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Row ID'}, {'feature_label': 'Row ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Row ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Segment', 'transformation_label': 'String Indexer'}], 'feature': 'Segment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '5077.17', 'stddev': '2902.52', 'min': ' 30/Box"', 'max': 'West', 'missing': '0', 'distinct': '238'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Segment'}, {'feature_label': 'Segment', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Segment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Ship Date', 'transformation_label': 'String Indexer'}], 'feature': 'Ship Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '4683.5', 'stddev': '3354.13', 'min': '1/1/2014', 'max': 'West', 'missing': '0', 'distinct': '1299'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Ship Date'}, {'feature_label': 'Ship Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Ship Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Ship Mode', 'transformation_label': 'String Indexer'}], 'feature': 'Ship Mode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '5379.55', 'stddev': '2762.92', 'min': '1/1/2017', 'max': 'Standard Class', 'missing': '0', 'distinct': '217'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Ship Mode'}, {'feature_label': 'Ship Mode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Ship Mode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '', 'stddev': '', 'min': '11/17/2014', 'max': 'Wisconsin', 'missing': '0', 'distinct': '62'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sub-Category', 'transformation_label': 'String Indexer'}], 'feature': 'Sub-Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '', 'stddev': '', 'min': '12/1/2015', 'max': 'Wisconsin', 'missing': '0', 'distinct': '66'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sub-Category'}, {'feature_label': 'Sub-Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sub-Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Top N Customer Label', 'transformation_label': 'String Indexer'}], 'feature': 'Top N Customer Label', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '', 'stddev': '', 'min': 'Binders', 'max': 'Washington', 'missing': '0', 'distinct': '19'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Top N Customer Label'}, {'feature_label': 'Top N Customer Label', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Top N Customer Label')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Discount', 'transformation_label': 'String Indexer'}], 'feature': 'Discount', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '0.16', 'stddev': '0.21', 'min': '0', 'max': 'Washington', 'missing': '0', 'distinct': '23'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Discount'}, {'feature_label': 'Discount', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Discount')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Number of Records', 'transformation_label': 'String Indexer'}], 'feature': 'Number of Records', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '0.98', 'stddev': '0.14', 'min': '0', 'max': 'Top 20 Customers', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Number of Records'}, {'feature_label': 'Number of Records', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Number of Records')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Profit', 'transformation_label': 'String Indexer'}], 'feature': 'Profit', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7995', 'mean': '24.3', 'stddev': '218.28', 'min': '-0.0895', 'max': 'Top 20 Customers', 'missing': '0', 'distinct': '5897'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Profit'}, {'feature_label': 'Profit', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Profit')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run gettingdataHooks.ipynb
try:
	#sourcePreExecutionHook()

	profitabilitydataset = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Profitabilitydataset2013-2016.csv', 'filename': 'Profitabilitydataset2013-2016.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run gettingdataHooks.ipynb
try:
	#transformationPreExecutionHook()

	generatingdatasetnew = TransformationMain.run(profitabilitydataset,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Category", "transformation_label": "String Indexer"}], "feature": "Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "", "stddev": "", "min": "Furniture", "max": "Technology", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Category"}, {"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "", "stddev": "", "min": "Aberdeen", "max": "Yuma", "missing": "0", "distinct": "503"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "Country", "transformation_label": "String Indexer"}], "feature": "Country", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "", "stddev": "", "min": "United States", "max": "United States", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Country"}, {"transformationsData": [{"feature_label": "Customer ID", "transformation_label": "String Indexer"}], "feature": "Customer ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "", "stddev": "", "min": "AA-10315", "max": "ZD-21925", "missing": "0", "distinct": "715"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer ID"}, {"transformationsData": [{"feature_label": "Customer Name", "transformation_label": "String Indexer"}], "feature": "Customer Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "", "stddev": "", "min": "Aaron Bergman", "max": "Zuschuss Donatelli", "missing": "0", "distinct": "715"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer Name"}, {"transformationsData": [{"feature_label": "Order Date", "transformation_label": "String Indexer"}], "feature": "Order Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "", "stddev": "", "min": "1/1/2016", "max": "9/9/2016", "missing": "0", "distinct": "1196"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Order Date"}, {"transformationsData": [{"feature_label": "Order ID", "transformation_label": "String Indexer"}], "feature": "Order ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "", "stddev": "", "min": "CA-2011-100006", "max": "US-2014-169551", "missing": "0", "distinct": "4092"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Order ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Postal Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7995", "mean": "55282.74", "stddev": "32018.49", "min": "1453", "max": "99301", "missing": "0"}, "updatedLabel": "Postal Code"}, {"transformationsData": [{"feature_label": "Product ID", "transformation_label": "String Indexer"}], "feature": "Product ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "", "stddev": "", "min": "FUR-BO-10000112", "max": "TEC-PH-10004977", "missing": "0", "distinct": "1821"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product ID"}, {"transformationsData": [{"feature_label": "Product Name", "transformation_label": "String Indexer"}], "feature": "Product Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "", "stddev": "", "min": "\"\"\"While you Were Out\"\" Message Book", "max": "netTALK DUO VoIP Telephone Service", "missing": "0", "distinct": "1803"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product Name"}, {"transformationsData": [{"feature_label": "Region", "transformation_label": "String Indexer"}], "feature": "Region", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "", "stddev": "", "min": " 10/Pack\"", "max": "West", "missing": "0", "distinct": "43"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Region"}, {"transformationsData": [{"feature_label": "Row ID", "transformation_label": "String Indexer"}], "feature": "Row ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "4983.42", "stddev": "2891.66", "min": " 1040 sheets\"", "max": "West", "missing": "0", "distinct": "7754"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Row ID"}, {"transformationsData": [{"feature_label": "Segment", "transformation_label": "String Indexer"}], "feature": "Segment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "5077.17", "stddev": "2902.52", "min": " 30/Box\"", "max": "West", "missing": "0", "distinct": "238"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Segment"}, {"transformationsData": [{"feature_label": "Ship Date", "transformation_label": "String Indexer"}], "feature": "Ship Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "4683.5", "stddev": "3354.13", "min": "1/1/2014", "max": "West", "missing": "0", "distinct": "1299"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Ship Date"}, {"transformationsData": [{"feature_label": "Ship Mode", "transformation_label": "String Indexer"}], "feature": "Ship Mode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "5379.55", "stddev": "2762.92", "min": "1/1/2017", "max": "Standard Class", "missing": "0", "distinct": "217"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "", "stddev": "", "min": "11/17/2014", "max": "Wisconsin", "missing": "0", "distinct": "62"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"feature_label": "Sub-Category", "transformation_label": "String Indexer"}], "feature": "Sub-Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "", "stddev": "", "min": "12/1/2015", "max": "Wisconsin", "missing": "0", "distinct": "66"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sub-Category"}, {"transformationsData": [{"feature_label": "Top N Customer Label", "transformation_label": "String Indexer"}], "feature": "Top N Customer Label", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "", "stddev": "", "min": "Binders", "max": "Washington", "missing": "0", "distinct": "19"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Top N Customer Label"}, {"transformationsData": [{"feature_label": "Discount", "transformation_label": "String Indexer"}], "feature": "Discount", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "0.16", "stddev": "0.21", "min": "0", "max": "Washington", "missing": "0", "distinct": "23"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Discount"}, {"transformationsData": [{"feature_label": "Number of Records", "transformation_label": "String Indexer"}], "feature": "Number of Records", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "0.98", "stddev": "0.14", "min": "0", "max": "Top 20 Customers", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Number of Records"}, {"transformationsData": [{"feature_label": "Profit", "transformation_label": "String Indexer"}], "feature": "Profit", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7995", "mean": "24.3", "stddev": "218.28", "min": "-0.0895", "max": "Top 20 Customers", "missing": "0", "distinct": "5897"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Profit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Profit Ratio", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "7995", "mean": "0.16", "stddev": "6.02", "min": "-292.0", "max": "157.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Profit Ratio"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Quantity", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "7995", "mean": "3.69", "stddev": "2.66", "min": "-38.0", "max": "106.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Quantity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sales", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "7995", "mean": "218.03", "stddev": "599.61", "min": "-1.55", "max": "22638.48", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Sales"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sales Per Customer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "7995", "mean": "219.36", "stddev": "599.39", "min": "0.0", "max": "22638.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Sales Per Customer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total Number of Customer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "7995", "mean": "3.21", "stddev": "25.11", "min": "1.0", "max": "1082.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Total Number of Customer"}]}))

	#transformationPostExecutionHook(generatingdatasetnew)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run gettingdataHooks.ipynb
try:
	#sinkPreExecutionHook()

	profitdatasetanalysisoutput = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/65c9937430d0571d2c3eac6c/part-00000-49ea688d-2191-4f68-bbbf-df3f56cc8195-c000.csv', 'samplecount': 917, 'originalcount': 9994, 'orignalKey': '/FileStore/platform/uploadedSourceFiles/Profitabilitydataset2013-2016.csv', 'pathOnly': '/Profit Dataset analysis', 'project_id': '65c411b1c138b7800690ac0f', 'parent_id': '65c411b1c138b7800690ac0f', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead6a', 'field': 'Postal Code', 'alias': 'Postal Code', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead6b', 'field': 'Profit Ratio', 'alias': 'Profit Ratio', 'type': 'real', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead6c', 'field': 'Quantity', 'alias': 'Quantity', 'type': 'real', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead6d', 'field': 'Sales', 'alias': 'Sales', 'type': 'real', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead6e', 'field': 'Sales Per Customer', 'alias': 'Sales Per Customer', 'type': 'real', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead6f', 'field': 'Total Number of Customer', 'alias': 'Total Number of Customer', 'type': 'real', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead70', 'field': 'Category_stringindexer', 'alias': 'Category_stringindexer', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead71', 'field': 'City_stringindexer', 'alias': 'City_stringindexer', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead72', 'field': 'Country_stringindexer', 'alias': 'Country_stringindexer', 'type': 'numeric', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead73', 'field': 'Customer ID_stringindexer', 'alias': 'Customer ID_stringindexer', 'type': 'real', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead74', 'field': 'Customer Name_stringindexer', 'alias': 'Customer Name_stringindexer', 'type': 'real', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead75', 'field': 'Order Date_stringindexer', 'alias': 'Order Date_stringindexer', 'type': 'real', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead76', 'field': 'Order ID_stringindexer', 'alias': 'Order ID_stringindexer', 'type': 'real', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead77', 'field': 'Product ID_stringindexer', 'alias': 'Product ID_stringindexer', 'type': 'real', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead78', 'field': 'Product Name_stringindexer', 'alias': 'Product Name_stringindexer', 'type': 'real', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead79', 'field': 'Region_stringindexer', 'alias': 'Region_stringindexer', 'type': 'real', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead7a', 'field': 'Row ID_stringindexer', 'alias': 'Row ID_stringindexer', 'type': 'real', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead7b', 'field': 'Segment_stringindexer', 'alias': 'Segment_stringindexer', 'type': 'real', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead7c', 'field': 'Ship Date_stringindexer', 'alias': 'Ship Date_stringindexer', 'type': 'real', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead7d', 'field': 'Ship Mode_stringindexer', 'alias': 'Ship Mode_stringindexer', 'type': 'real', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead7e', 'field': 'State_stringindexer', 'alias': 'State_stringindexer', 'type': 'real', 'position': '20'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead7f', 'field': 'Sub-Category_stringindexer', 'alias': 'Sub-Category_stringindexer', 'type': 'real', 'position': '21'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead80', 'field': 'Top N Customer Label_stringindexer', 'alias': 'Top N Customer Label_stringindexer', 'type': 'real', 'position': '22'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead81', 'field': 'Discount_stringindexer', 'alias': 'Discount_stringindexer', 'type': 'real', 'position': '23'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead82', 'field': 'Number of Records_stringindexer', 'alias': 'Number of Records_stringindexer', 'type': 'real', 'position': '24'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead83', 'field': 'Profit_stringindexer', 'alias': 'Profit_stringindexer', 'type': 'real', 'position': '25'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead50', 'field': 'Category', 'alias': 'Category', 'type': 'String', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead51', 'field': 'City', 'alias': 'City', 'type': 'String', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead52', 'field': 'Country', 'alias': 'Country', 'type': 'String', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead53', 'field': 'Customer ID', 'alias': 'Customer ID', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead54', 'field': 'Customer Name', 'alias': 'Customer Name', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead55', 'field': 'Order Date', 'alias': 'Order Date', 'type': 'date', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead56', 'field': 'Order ID', 'alias': 'Order ID', 'type': 'String', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead57', 'field': 'Postal Code', 'alias': 'Postal Code', 'type': 'numeric', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead58', 'field': 'Product ID', 'alias': 'Product ID', 'type': 'String', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead59', 'field': 'Product Name', 'alias': 'Product Name', 'type': 'String', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead5a', 'field': 'Region', 'alias': 'Region', 'type': 'String', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead5b', 'field': 'Row ID', 'alias': 'Row ID', 'type': 'String', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead5c', 'field': 'Segment', 'alias': 'Segment', 'type': 'String', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead5d', 'field': 'Ship Date', 'alias': 'Ship Date', 'type': 'String', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead5e', 'field': 'Ship Mode', 'alias': 'Ship Mode', 'type': 'String', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead5f', 'field': 'State', 'alias': 'State', 'type': 'String', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead60', 'field': 'Sub-Category', 'alias': 'Sub-Category', 'type': 'String', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead61', 'field': 'Top N Customer Label', 'alias': 'Top N Customer Label', 'type': 'String', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead62', 'field': 'Discount', 'alias': 'Discount', 'type': 'String', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead63', 'field': 'Number of Records', 'alias': 'Number of Records', 'type': 'String', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead64', 'field': 'Profit', 'alias': 'Profit', 'type': 'String', 'position': '20'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead65', 'field': 'Profit Ratio', 'alias': 'Profit Ratio', 'type': 'real', 'position': '21'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead66', 'field': 'Quantity', 'alias': 'Quantity', 'type': 'real', 'position': '22'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead67', 'field': 'Sales', 'alias': 'Sales', 'type': 'real', 'position': '23'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead68', 'field': 'Sales Per Customer', 'alias': 'Sales Per Customer', 'type': 'real', 'position': '24'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c995f530d0571d2c3ead69', 'field': 'Total Number of Customer', 'alias': 'Total Number of Customer', 'type': 'real', 'position': '25'}], 'server': 'https://ml.colaberry.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'FILERESULTSNEW.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-e470da56-6d38-4f73-84b6-6cd057673d3e-c000.csv', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
